# Prerequisites

This converter relies on Docker for the abstraction of the environment. You can install Docker from [here](https://docs.docker.com/install/).

The used console commands require Bash.

In [1]:
import os

# Install via pip: pyproj
import sys
!{sys.executable} -m pip install pyproj
from pyproj import Transformer, CRS

import xml.etree.ElementTree as ET

Defaulting to user installation because normal site-packages is not writeable


# Download OpenStreetMaps data for Schleswig-Holstein

In [2]:
# Download from http://download.geofabrik.de/europe/germany/schleswig-holstein-latest.osm.pbf if file does not exist in /input folder
if not os.path.isfile('./input/schleswig-holstein-latest.osm.pbf'):
    os.system('wget http://download.geofabrik.de/europe/germany/schleswig-holstein-latest.osm.pbf -P /input')
    print('Downloaded schleswig-holstein-latest.osm.pbf from http://download.geofabrik.de/europe/germany/schleswig-holstein-latest.osm.pbf')
else :
    print('schleswig-holstein-latest.osm.pbf already exists in /input folder')

# Print file size and MD5 hash
print('MD5 hash of schleswig-holstein-latest.osm.pbf:')
os.system('md5sum ./input/schleswig-holstein-latest.osm.pbf')
print('File size of schleswig-holstein-latest.osm.pbf:')
os.system('ls -lh ./input/schleswig-holstein-latest.osm.pbf')

# Print green coloured success message
print('\033[92m' + 'Download of schleswig-holstein-latest.osm.pbf completed' + '\033[0m')

schleswig-holstein-latest.osm.pbf already exists in /input folder
MD5 hash of schleswig-holstein-latest.osm.pbf:
b5c67d5f036a33d87d8a054048dc139b  ./input/schleswig-holstein-latest.osm.pbf
File size of schleswig-holstein-latest.osm.pbf:
-rw-r--r-- 1 armin armin 136M Jan 20 03:29 ./input/schleswig-holstein-latest.osm.pbf
Download of schleswig-holstein-latest.osm.pbf completed


# Find bounding box for the area to be converted

The bounding box coordinates need to be provided in the OSM coordinate system.
You can find the bounding box coordinates for your area of interest at the following websites:
- [Openstreetmap.org](https://www.openstreetmap.org/export#map=10/54.1870/9.9930): Zoom to your area of interest and copy the coordinates shown on the left side of the screen.
- [boundingbox.klokantech.com](https://boundingbox.klokantech.com/): Select the area of interest with the movable box and use DublinCore on the bottom left side as the format to get the coordinates in an easily readable format.

In [3]:
# Set bounding box for Kiel (include Strande!)
top=54.4896
left=9.9152
bottom=54.1938
right=10.4343

# Reduce OSM data to the area of interest and remove unnecessary data using Osmosis

Detailed information about the parameters used can be found in the [OSM Wiki](https://wiki.openstreetmap.org/wiki/Osmosis/Detailed_Usage_0.48).

In [4]:
# Execute Java Osmosis command to extract data from bounding box
os.system('./vendor/osmosis/bin/osmosis --rb file=./input/schleswig-holstein-latest.osm.pbf --bounding-box top=' + str(top) + ' left=' + str(left) + ' bottom=' + str(bottom) + ' right=' + str(right) + ' --used-node --write-xml ./input/kiel.osm')

Mar 11, 2024 12:26:14 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Osmosis Version 0.46
Mar 11, 2024 12:26:15 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Preparing pipeline.
Mar 11, 2024 12:26:15 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Launching pipeline execution.
Mar 11, 2024 12:26:15 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Pipeline executing, waiting for completion.
Mar 11, 2024 12:26:42 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Pipeline complete.
Mar 11, 2024 12:26:42 PM org.openstreetmap.osmosis.core.Osmosis run
INFO: Total execution time: 28100 milliseconds.


0

# Convert reduced OSM dataset to MATSim network

## Build Docker image

Information on the deprecation message: https://docs.docker.com/engine/deprecated/#legacy-builder-fallback

In [5]:
os.system('./bin/build.sh')

failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/



Sending build context to Docker daemon  120.2MB
Step 1/6 : FROM eclipse-temurin:8-jdk-jammy
 ---> c05a9a5ad465
Step 2/6 : RUN mkdir /osm2matsim
 ---> Running in ddd018c1e945
 ---> Removed intermediate container ddd018c1e945
 ---> c684e583cc99
Step 3/6 : WORKDIR /osm2matsim
 ---> Running in 54f1f9e31ba8
 ---> Removed intermediate container 54f1f9e31ba8
 ---> dd80b794acec
Step 4/6 : COPY . .
 ---> 6cf2c2fdd7ed
Step 5/6 : RUN javac -cp vendor/matsim-0.9.0/matsim-0.9.0.jar Osm2matsim.java
 ---> Running in 273e8c902f6f
 ---> Removed intermediate container 273e8c902f6f
 ---> 560030f0e0cb
Step 6/6 : ENTRYPOINT ["java", "-cp", "vendor/matsim-0.9.0/matsim-0.9.0.jar:.", "Osm2matsim"]
 ---> Running in bee906af76b8
 ---> Removed intermediate container bee906af76b8
 ---> 4f280cda339c
Successfully built 4f280cda339c
Successfully tagged osm2matsim:latest


0

## Convert using the Docker image

In [6]:
os.system('./bin/convert.sh input/kiel.osm output/kiel.xml')

Converting input/kiel.osm into output/kiel.xml


2024-03-11 11:27:19,267  WARN MGC:175 Assuming that coordinates are in longitude first notation, i.e. (longitude, latitude).


2024-03-11 11:27:19,783  INFO Logger:? dataFileCache open start
2024-03-11 11:27:20,104  INFO Config:131 context=[file:/osm2matsim/]
2024-03-11 11:27:20,168  INFO Config:131 context=[file:/osm2matsim/]
2024-03-11 11:27:20,187  INFO OsmNetworkReader:130 Falling back to default values.
2024-03-11 11:27:20,190  INFO MatsimXmlParser:147 starting to parse xml from file input/kiel.osm ...
2024-03-11 11:27:20,206  INFO Counter:61 node 1
2024-03-11 11:27:20,207  INFO Counter:61 node 2
2024-03-11 11:27:20,208  INFO Counter:61 node 4
2024-03-11 11:27:20,209  INFO Counter:61 node 8
2024-03-11 11:27:20,211  INFO Counter:61 node 16
2024-03-11 11:27:20,214  INFO Counter:61 node 32
2024-03-11 11:27:20,220  INFO Counter:61 node 64
2024-03-11 11:27:20,227  INFO Counter:61 node 128
2024-03-11 11:27:20,244  INFO Counter:61 node 256
2024-03-11 11:27:20,269  INFO Counter:61 node 512
2024-03-11 11:27:20,296  INFO Counter:61 node 1024


2024-03-11 11:27:20,188  WARN OsmNetworkReader:190 No hierarchy layer specified. Will convert every highway specified by setHighwayDefaults.


2024-03-11 11:27:20,341  INFO Counter:61 node 2048
2024-03-11 11:27:20,393  INFO Counter:61 node 4096
2024-03-11 11:27:20,499  INFO Counter:61 node 8192
2024-03-11 11:27:20,589  INFO Counter:61 node 16384
2024-03-11 11:27:20,684  INFO Counter:61 node 32768
2024-03-11 11:27:20,786  INFO Counter:61 node 65536
2024-03-11 11:27:20,983  INFO Counter:61 node 131072
2024-03-11 11:27:21,387  INFO Counter:61 node 262144
2024-03-11 11:27:22,216  INFO Counter:61 node 524288
2024-03-11 11:27:30,687  INFO Counter:61 node 1048576
2024-03-11 11:27:31,138  INFO Counter:61 way 1
2024-03-11 11:27:31,139  INFO Counter:61 way 2
2024-03-11 11:27:31,140  INFO Counter:61 way 4
2024-03-11 11:27:31,142  INFO Counter:61 way 8
2024-03-11 11:27:31,145  INFO Counter:61 way 16
2024-03-11 11:27:31,148  INFO Counter:61 way 32
2024-03-11 11:27:31,150  INFO Counter:61 way 64
2024-03-11 11:27:31,155  INFO Counter:61 way 128
2024-03-11 11:27:31,168  INFO Counter:61 way 256
2024-03-11 11:27:31,184  INFO Counter:61 way 512

2024-03-11 11:27:34,924  WARN LinkImpl:140 [permlanes=0.5 of link id 126 may cause problems]
2024-03-11 11:27:34,924  WARN LinkImpl:142  Future occurences of this logging statement are suppressed.
2024-03-11 11:27:34,970  WARN OsmNetworkReader:539 Could not parse maxspeed tag:For input string: "walk". Ignoring it.
2024-03-11 11:27:34,971  WARN OsmNetworkReader:539 Could not parse maxspeed tag:For input string: "none". Ignoring it.


2024-03-11 11:27:35,075  INFO NetworkImpl:152  link # 32768
2024-03-11 11:27:35,076  INFO OsmNetworkReader:465 ... done creating the links.
2024-03-11 11:27:35,080  INFO OsmNetworkReader:221 = conversion statistics: ==========================
2024-03-11 11:27:35,081  INFO OsmNetworkReader:222 osm: # nodes read:       1186005
2024-03-11 11:27:35,081  INFO OsmNetworkReader:223 osm: # ways read:        13665
2024-03-11 11:27:35,081  INFO OsmNetworkReader:224 MATSim: # nodes created: 15573
2024-03-11 11:27:35,081  INFO OsmNetworkReader:225 MATSim: # links created: 32920
2024-03-11 11:27:35,082  INFO OsmNetworkReader:233 = end of conversion statistics ====================
2024-03-11 11:27:35,083  INFO NetworkCleaner:118 running org.matsim.core.network.algorithms.NetworkCleaner algorithm...
2024-03-11 11:27:35,083  INFO NetworkCleaner:121   checking 15573 nodes and 32920 links for dead-ends...
2024-03-11 11:27:35,159  INFO NetworkCleaner:139     The biggest cluster consists of 15399 nodes.
2

0

# Convert coordinates in MATSim network to the coordinate system used by the simulation

The conversion is based on the coordinate system set for the TransformationFactory in Osm2matsim.java.
If using the following line, the code below will work without any changes:
```
CoordinateTransformation ct = TransformationFactory.getCoordinateTransformation(TransformationFactory.WGS84, TransformationFactory.WGS84_UTM33N);
```

In [7]:
import sys
import time
from tqdm import tqdm
# Load the XML file
xml_file = "./output/kiel.xml"
tree = ET.parse(xml_file)
root = tree.getroot()

# Define coordinate transformation functions
def epsg32633_to_epsg25832(lon, lat):
    transformer = Transformer.from_crs(CRS.from_epsg(32633), CRS.from_epsg(25832), always_xy=True)
    x, y = transformer.transform(lon, lat)
    return x, y

# Get the total number of nodes
total_nodes = len(root.findall('.//node'))

# Parse and update node coordinates
progress_step = total_nodes // 10  # 10% steps
for i, node in enumerate(tqdm(root.findall('.//node'), total=total_nodes, desc='Converting coordinates')):
    lon = float(node.attrib['x'])
    lat = float(node.attrib['y'])
    x, y = epsg32633_to_epsg25832(lon, lat)
    node.attrib['x'] = str(x)
    node.attrib['y'] = str(y)
    
    # Print progress in 10% steps
    if i % progress_step == 0:
        progress = (i + 1) / total_nodes * 100
        tqdm.write(f"Progress: {progress:.2f}%")

# Save the modified XML file
output_file = "./output/network_kiel_osm_converted_EPSG25832.xml"
# Ensure custom DOCTYPE is used: <!DOCTYPE network SYSTEM "http://www.matsim.org/files/dtd/network_v2.dtd">
# Otherwise, MATSim will not be able to read the file
with open(output_file, 'wb') as f:
    f.write('<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE network SYSTEM "http://www.matsim.org/files/dtd/network_v2.dtd">'.encode('utf8'))
    tree.write(f, encoding='utf-8', xml_declaration=False)
    
print(f"Coordinate conversion completed. Output saved to {output_file}")


Converting coordinates:   0%|          | 8/15399 [00:00<06:32, 39.18it/s]

Progress: 0.01%


Converting coordinates:  10%|█         | 1546/15399 [00:33<05:26, 42.49it/s]

Progress: 10.00%


Converting coordinates:  20%|██        | 3086/15399 [01:04<04:06, 49.96it/s]

...

Converting coordinates:  90%|████████▉ | 13859/15399 [04:50<00:32, 47.21it/s]

Progress: 89.95%


Converting coordinates: 100%|██████████| 15399/15399 [05:21<00:00, 47.85it/s]


Progress: 99.95%


TypeError: ElementTree.write() got an unexpected keyword argument 'doctype'

# Preview network

To preview the network, you can use the MATSim network editor:
``` ./bin/edit_network.sh ```